In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import math

Entrenar modelo Naive Bayes con tablas de frecuencia

In [ ]:
def entrenar_naive_bayes(datos_entrenamiento, idx_clase, dominio, cabecera, tipos):

    tabla_frecuencia = {}

    for idx_col, atributo in enumerate(cabecera.iloc[0]):
        tabla_frecuencia[atributo] = {}
        columna = datos_entrenamiento.iloc[:, idx_col]

        if idx_col == idx_clase:
            for idx, valor in columna.items():
                valor_clase = datos_entrenamiento.iloc[idx, idx_clase]
                if valor_clase not in tabla_frecuencia[atributo]:
                    tabla_frecuencia[atributo][valor_clase] = {}
                if valor not in tabla_frecuencia[atributo][valor_clase]:
                    tabla_frecuencia[atributo][valor_clase][valor] = 0
                tabla_frecuencia[atributo][valor_clase][valor] += 1
        else:
            for clase in dominio.iloc[0, idx_clase].split(' '):
                if clase not in tabla_frecuencia[atributo]:
                    tabla_frecuencia[atributo][clase] = {}
                    for idx, valor in columna.items():
                        if valor not in tabla_frecuencia[atributo][clase]:
                            tabla_frecuencia[atributo][clase][valor] = 0

                        valor_clase = datos_entrenamiento.iloc[idx, idx_clase]
                        if valor_clase == clase:
                            tabla_frecuencia[atributo][clase][valor] += 1


    # Verosimilitud
    cabecera_cont = [cabecera.iloc[0, idx] for idx in range(len(cabecera.columns)) if tipos.iloc[0, idx] == 'continuous']

    for atributo, clases in tabla_frecuencia.items():
        for clase, valores in clases.items():
            total = sum(valores.values())
            if atributo in cabecera_cont:
                # Para atributos continuos, calcular media y desviación estándar
                freq_dict = valores
                all_values = []
                for valor, count in freq_dict.items():
                    all_values.extend([float(valor)] * int(count))
                all_values = pd.Series(all_values)
                media = all_values.mean()
                std_dev = all_values.std(ddof=0)
                tabla_frecuencia[atributo][clase] = {'mean': media, 'std': std_dev}
            else:
                idx_atributo = cabecera.iloc[0].tolist().index(atributo)

                for valor, count in valores.items():
                    if count == 0:
                        # Aplicar suavizado de Laplace
                        for value, count in valores.items():
                            tabla_frecuencia[atributo][clase][value] += 1
                            tabla_frecuencia[atributo][clase][value] /= (total + len(valores))

                for valor, count in valores.items():
                    if not isinstance(count, float):
                        if valor in list(dominio[idx_clase])[0].split(' '):
                                tabla_frecuencia[atributo][clase][valor] = count / len(datos_entrenamiento)
                        else:
                            tabla_frecuencia[atributo][clase][valor] = count / total

    return tabla_frecuencia

Pedecir usando Naive Bayes

In [ ]:
def clasificar_naive_bayes(instancia, tabla_frecuencia, cabecera, tipos, dominio, idx_clase):
    cabecera_cont = [cabecera.iloc[0, idx] for idx in range(len(cabecera.columns)) if tipos.iloc[0, idx] == 'continuous']
    probabilidades = {}
    for clases in dominio.iloc[0, idx_clase].split(' '):
        probabilidades[clases] = 1

    for atributo, clases in tabla_frecuencia.items():
        for clase, valores in clases.items():
            probabilidad_A_I = 1
            probabilidad_A = 0
            valor = instancia[cabecera.iloc[0].tolist().index(atributo)]
            if atributo in cabecera_cont:
                media = valores['mean']
                std_dev = valores['std']
                probabilidad_A_I *= (1 / (math.sqrt(2 * math.pi) * std_dev)) * math.exp(-((float(valor) - media) ** 2) / (2 * (std_dev ** 2)))
                if probabilidad_A_I == 0:
                    probabilidad_A_I = 1e-5
            elif next(iter(valores)) == clase:
                probabilidad_A = valores.get(clase, 0)
            else:
                probabilidad_A_I *= valores.get(valor, 0)

            if probabilidad_A != 0:
                probabilidades[clase] *= probabilidad_A
            else:
                probabilidades[clase] *= probabilidad_A_I

    return probabilidades

Validacion iterativa de K-Fold Cross

In [ ]:
def k_fold_cross_validation(datos, idx_clase, dominio, cabecera, tipos, K):

    indices = np.arange(len(datos))
    np.random.shuffle(indices)
    folds = np.array_split(indices, K)
    model_accuracies = []

    for i in range(K):
        test_indices = folds[i]
        train_indices = np.concatenate(folds[:i] + folds[i+1:])

        datos_entrenamiento = datos.iloc[train_indices].reset_index(drop=True)
        datos_prueba = datos.iloc[test_indices].reset_index(drop=True)

        tabla_frecuencia = entrenar_naive_bayes(datos_entrenamiento, idx_clase, dominio, cabecera, tipos)

        aciertos = 0
        for idx, instancia in datos_prueba.iterrows():
            probabilidades = clasificar_naive_bayes(instancia, tabla_frecuencia, cabecera, tipos, dominio, idx_clase)
            clase_predicha = max(probabilidades, key=probabilidades.get)
            if clase_predicha == instancia[idx_clase]:
                aciertos += 1

        accuracy = aciertos / len(datos_prueba)
        model_accuracies.append((tabla_frecuencia, accuracy))

    # Calcular accuracy promedio
    avg_accuracy = sum(x[1] for x in model_accuracies) / len(model_accuracies)
    # Seleccionar el modelo cuyo accuracy esté más cercano al promedio
    best_model = min(model_accuracies, key=lambda x: abs(x[1] - avg_accuracy))

    #print("Accuracy promedio:", avg_accuracy)
    #print("Modelo seleccionado con accuracy:", best_model[1])

    return best_model  # Retorna una tupla (tabla_frecuencia, accuracy)

Metodo Holdout

In [ ]:
def holdout(datos, training_percentage, K, dominio, cabecera, tipos, idx_clase):
    # Dividir el conjunto de datos en entrenamiento y prueba
    indices = np.arange(len(datos))
    np.random.shuffle(indices)
    split_point = int(len(datos) * training_percentage)
    train_indices = indices[:split_point]
    test_indices = indices[split_point:]
    training_set = datos.iloc[train_indices].reset_index(drop=True)
    test_set = datos.iloc[test_indices].reset_index(drop=True)

    best_model = k_fold_cross_validation(training_set, idx_clase, dominio, cabecera, tipos, K)


    clases = list(dominio.iloc[0, idx_clase].split(' '))

    cm = {clase: {c: 0 for c in clases} for clase in clases}

    for _, instancia in test_set.iterrows():
        predicted_prob = clasificar_naive_bayes(instancia, best_model[0], cabecera, tipos, dominio, idx_clase)
        predicted_class = max(predicted_prob, key=predicted_prob.get)
        actual_class = instancia[idx_clase]
        cm[actual_class][predicted_class] += 1
        # Acumular los detalles de cada instancia en una lista (inicializándola si es la primera iteración)
        if 'tabla_resultados' not in locals():
            tabla_resultados = []
        tabla_resultados.append({
            "Instancia": instancia.to_dict(),
            "Clase Actual": actual_class,
            "Clase Predicha": predicted_class,
            "Probabilidades": predicted_prob
        })

        # Si ya se han procesado todas las instancias del conjunto de prueba, mostrar la tabla
        if len(tabla_resultados) == len(test_set):
            resultados_df = pd.DataFrame(tabla_resultados)
            print("Tabla de resultados:")
            tabla_formateada = pd.DataFrame({
                'Instancia': [str(inst) for inst in resultados_df['Instancia']],
                'Clase Actual': resultados_df['Clase Actual'],
                'Clase Predicha': resultados_df['Clase Predicha'],
                'Probabilidades': [str(prob) for prob in resultados_df['Probabilidades']]
            })

            print(tabla_formateada.to_string(index=True))
    # Mostrar la matriz de confusión
    print("Matriz de confusión:")
    matrix = pd.DataFrame(cm)
    matrix.index.name = "Clase Actual"
    matrix.columns.name = "Clase Predicho"
    print(matrix)

    return cm

In [ ]:
def main():
    # Configuración hardcoded
    filename = "/content/drive/MyDrive/Colab Notebooks/iris_mixed.csv"
    N = 10  # Numero de iteraciones
    K = 5 # Numero de Folds
    training_percentage = 0.7

    # Cargar datos
    data = pd.read_csv(filename, header = None)
    cabecera = data.iloc[[0]].reset_index(drop=True)
    dominio = data.iloc[[1]].reset_index(drop=True)
    tipos = data.iloc[[2]].reset_index(drop=True)
    datos = data.iloc[3:].reset_index(drop=True)
    idx_clase = tipos.columns[tipos.iloc[0] == 'class'][0]

    all_confusion_matrices = []
    clases = list(dominio.iloc[0, idx_clase].split(' '))

    for iteration in range(N):
        print("\nIteración:", iteration + 1)

        all_confusion_matrices.append(holdout(datos, training_percentage, K, dominio, cabecera, tipos, idx_clase))

    # Calcular la matriz promedio (promediando cada componente)
    avg_confusion = {}
    for clase in clases:
        avg_confusion[clase] = {}
        for c in clases:
            avg_confusion[clase][c] = np.mean([matriz[clase][c] for matriz in all_confusion_matrices])

    print("-" * 50)
    print("Matriz de confusión promedio:")
    matrix = pd.DataFrame(avg_confusion)
    matrix.index.name = "Clase Actual"
    matrix.columns.name = "Clase Predicho"
    print(matrix)

    # Calcular las métricas a partir de la matriz promedio
    total = sum(avg_confusion[a][p] for a in clases for p in clases)
    exactitud = sum(avg_confusion[c][c] for c in clases) / total if total else 0

    sensibilidades = {}
    especificidades = {}
    f1_scores = {}
    precisiones = {}
    for c in clases:
        TP = avg_confusion[c][c]
        TN = sum(avg_confusion[other][other] for other in clases if other != c)
        FP = sum(avg_confusion[other][c] for other in clases if other != c)
        FN = total - TP - FP - TN
        P = sum(avg_confusion[c][other] for other in clases)
        N = total - P

        sensibilidad = TP / P if P > 0 else 0
        especificidad = TN / N if N > 0 else 0
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        f1 = (2 * precision * sensibilidad) / (precision + sensibilidad) if (precision + sensibilidad) > 0 else 0
        sensibilidades[c] = sensibilidad
        especificidades[c] = especificidad
        f1_scores[c] = f1
        precisiones[c] = precision

    print("-" * 50)
    print("Métricas finales:")
    print("Exactitud (promedio):", exactitud)
    print("Sensibilidad (promedio):", sensibilidades)
    print("Especificidad (promedio):", especificidades)
    print("Precisión (promedio):", precisiones)
    print("F1-score (promedio):", f1_scores)
    print("-" * 50)

if __name__ == "__main__":
    main()


Iteración: 1
Tabla de resultados:
                                                                         Instancia     Clase Actual   Clase Predicha                                                                                                                 Probabilidades
0    {0: '6.4', 1: '(1.998, 2.8]', 2: '5.3', 3: '(1.7, 2.5]', 4: 'Iris-virginica'}   Iris-virginica   Iris-virginica       {'Iris-setosa': 5.22292018194055e-97, 'Iris-virginica': 0.042467488046429996, 'Iris-versicolor': 0.00016987788398333534}
1     {0: '5.4', 1: '(2.8, 3.6]', 2: '4.5', 3: '(0.9, 1.7]', 4: 'Iris-versicolor'}  Iris-versicolor  Iris-versicolor        {'Iris-setosa': 6.971503580747784e-59, 'Iris-virginica': 0.0003178636283275255, 'Iris-versicolor': 0.05012236800351265}
2     {0: '6.9', 1: '(2.8, 3.6]', 2: '4.9', 3: '(0.9, 1.7]', 4: 'Iris-versicolor'}  Iris-versicolor  Iris-versicolor       {'Iris-setosa': 1.2498611675350216e-80, 'Iris-virginica': 0.003818377249859368, 'Iris-versicolor': 0.012686339